# Scoping

Problem/Opportunity
With the growing decline in oat planting in Manitoba and Saskatchewan 1, 
a group of companies supplying oat grains needs to understand more 
precisely the best locations for crop oats and thus make better future investment decisions when production returns to normal.

Solution
Recommendation of best areas using K-means clustering for 
oat cultivation.

##  ETL (Extract, Transform, Load)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd  # command to tape on anaconda prompt:conda install -c conda-forge geopandas
import matplotlib.pyplot as plt
import seaborn as sb
import contextily as ctx
import folium
plt.rcParams['figure.figsize'] = [12, 8]

## Yield

In [ ]:
df_sk=pd.read_csv('/Users/robertotabosa/Desktop/Stream3/rm-yields-data.csv')

In [ ]:
df_mb=pd.read_excel('/Users/robertotabosa/Desktop/Stream3/MMPP - Yield by Soil Type Browser.xlsx')

## Shapefiles

In [ ]:
gdf_sk=gpd.read_file('/Users/robertotabosa/Desktop/Stream3/RuralMunicipality.shp') # command to tape on anaconda prompt:conda install -c conda-forge pyogrio

In [ ]:
gdf_mb=gpd.read_file('/Users/robertotabosa/Desktop/Stream3/MB_Municipal_Boundaries.shp')

## Understanding & Transforming

In [ ]:
# List of columns
df_sk.columns

In [ ]:
# List of columns
df_mb.columns

In [ ]:
# Info about columns
df_sk.info()

In [ ]:
# Info about columns
df_mb.info()

In [ ]:
# Table heads
df_sk.head()

In [ ]:
# Table heads
df_mb.head()

In [ ]:
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].str.replace(' Tonnes', '') # Replacing Tonnes
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].replace('Tolerance', np.NaN) # Replacing Tolerance
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].astype(float) # changing object to float data type
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('RM OF ','')
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('RM OF ','')
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('MUNICIPALITY OF ','', regex=False) # not caps sensitive\n",
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace(' MUNICIPALITY','', regex=False) # not caps sensitive\n",
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('CITY OF ','', regex=False) # not caps sensitive"

In [ ]:
df_mb_pivot=pd.pivot_table(df_mb.drop(columns=['Yield/acre(Metric).1', 'Yield/acre(Imperial)', 'Soil', 'Farms' ]),
               index=['Risk Area / R.M.', 'Year'], columns='Crop', values='Yield/acre(Metric)')\
               .reset_index()

In [ ]:
df_sk.columns

In [ ]:
df_mb_pivot.columns

In [ ]:
df_mb_clean=df_mb_pivot.rename(columns={
    'Risk Area / R.M.': 'RM', 
    'ARGENTINE CANOLA': 'Canola', 
    'BARLEY':'Barley',
    'CANARYSEED': 'Canary Seed', 
    'DURUM WHEAT': 'Durum Wheat', 
     'LENTILS': 'Lentils', 
     'OATS': 'Oats',
    'RED SPRING WHEAT': 'Spring Wheat', 
    'WHITE PEA BEANS': 'Peas'}) \
        .drop(columns=['ALFALFA', 'FABABEANS', 'FABABEANS', 'POLISH CANOLA'] ) # Some crops don't exist in the df_sk data frame and that's why were removed of df_mb.

In [ ]:
# Some crops don't exist in the df_mb data frame and that's why were removed of df_sk.
df_sk_clean=df_sk.rename(columns={'Durum': 'Durum Wheat'}).drop(columns=['Winter Wheat', 'Mustard', 'Sunflowers', 'Fall Rye', 'Spring Rye', 'Tame Hay','Flax', 'Chickpeas' ] )

In [ ]:
df_sk_clean

In [ ]:
df_mb_clean

In [ ]:
# Crop conversion in MB tonnes to bushel
df_mb_clean['Canola']=df_mb_clean['Canola'] * 44.092
df_mb_clean['Barley']=df_mb_clean['Barley'] * 45.93
df_mb_clean['Canary Seed']=df_mb_clean['Canary Seed'] * 44.092
df_mb_clean['Durum Wheat']=df_mb_clean['Durum Wheat'] * 36.74
df_mb_clean['Lentils']=df_mb_clean['Lentils'] * 36.74
df_mb_clean['Oats']=df_mb_clean['Oats'] * 64.842
df_mb_clean['Spring Wheat']=df_mb_clean['Spring Wheat'] * 36.74
df_mb_clean['Peas']=df_mb_clean['Peas'] *  36.74

In [ ]:
# Crop Conversion in SK - pounds to bushels
df_sk_clean['Lentils']=df_sk_clean['Lentils'] / 60
df_sk_clean['Canary Seed']=df_sk_clean['Canary Seed'] / 50

In [ ]:
# Creating province column
df_mb_clean['Province'] = 'MB'
df_sk_clean['Province'] = 'SK'

In [ ]:
df = pd.concat([df_mb_clean, df_sk_clean])

In [ ]:
df = df[df["Year"]>=2003] ## I chose to work with data from 2003 onwards.

In [ ]:
canola_null_count = df['Oats'].isnull().sum()
print("Number of null values in the 'Oats' column:", canola_null_count)

canola_total_count = len(df['Canola'])
print("Total number of records in the 'Canola' column:", canola_total_count)

## EDA (Exploratory Data Analysis)

### Missing Values

In [ ]:
# Get the counts of missing values for each column
missing_values_counts = df.loc[df['Year'] >= 2003].isna().sum().sort_values()

# Calculate the bar colors based on their heights, using shades of green
shades_of_green = np.linspace(0.2, 1, len(missing_values_counts))
bar_colors = plt.cm.Greens(shades_of_green)

# Plot the bar chart
ax = missing_values_counts.plot(kind='bar', color=bar_colors)
plt.title('Missing Values - 2003 to 2022')
plt.xlabel('Crops')
plt.ylabel('# of Missing values')

# Rotate x-axis labels by 45 degrees
plt.xticks(rotation=45)

# Add horizontal lines
plt.axhline(len(df) / 2, linestyle='--', color='red')
plt.axhline(len(df) / 4, linestyle='--', color='red')
plt.axhline(len(df) / 10, linestyle='--', color='red')

plt.show()


## Outliers

In [ ]:
df = df[df["Year"]>=2003]
# Remove rows where "Oats" column is null
df = df.dropna(subset=['Oats'])

In [ ]:
crop_columns=['Canola', 'Barley', 'Canary Seed', 'Durum Wheat',
       'Lentils', 'Oats', 'Spring Wheat', 'Peas']

## Histograms

In [ ]:
sb.histplot(df["Oats"], bins=300, color=sb.color_palette("YlOrBr")[0])
plt.show()

In [ ]:
sb.heatmap(df[crop_columns].corr(), annot=True)

## Boxplots

In [ ]:
df[crop_columns].boxplot()

In [ ]:
# Remove outliers: Delete lines to 'Oats'>70
df = df.loc[df['Oats']<155]
df.info() 

In [ ]:
# Peorson Corr (-1 to 1), -1 negative corr, o no corr, 1 positive corr
# Using Seaborn
sb.heatmap(df[crop_columns].corr(),annot=True, cmap='Greens')  ## deu errado apenas pq no codigo acima coloque df apenas com a coluna de Oats, proxima vez colocar o codigo acima abaixo desse

## GIS Analysis

In [ ]:
# CRS --> Cordinate Reference Systems
gdf_mb['geometry'].crs

In [ ]:
gdf_mb

In [ ]:
# Standardazing CRS formats

# Define the target CRS (EPSG:4326 - WGS 84)
target_crs = 'EPSG:4326'

# Transform the GeoDataFrame to the target CRS
gdf_mb = gdf_mb.to_crs(target_crs)
gdf_sk = gdf_sk.to_crs(target_crs)

# Now, gdf_transformed contains the GeoDataFrame in the new CRS (EPSG:4326)



In [ ]:
gdf_sk.explore()

In [ ]:
gdf_mb.explore()

In [ ]:
# Renaming column name
gdf_sk['RMNO']=gdf_sk['RMNO'].astype(int)   # Rural Municipality Number
gdf_sk.explore()

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])
gdf

In [ ]:
set(gdf["RM"].unique()) - set(df_sk_clean["RM"].unique())   #this command calculates the set difference of the unique 'RM' values between gdf and df_sk_clean, providing the 'RM' values that are unique to gdf.

In [ ]:
gdf_mb['MUNI_NAME']=gdf_mb['MUNI_NAME'].str.replace('RM OF ','')

gdf.loc[gdf["RM"] == "ST. FRANCOIS XAVIER", "RM"] = "ST. FRANCIS XAVIER"
gdf.loc[gdf["RM"] == "DE SALABERRY", "RM"] = "DESALABERRY"
df.loc[df["RM"] == "EAST ST PAUL", "RM"] = "EAST ST. PAUL"
df.loc[df["RM"] == "KILLARNEY-TURTLE MTN" "RM"] = "KILLARNEY-TURTLE MOUNTAIN"
gdf.loc[gdf["RM"] == "ROBLIN", "RM"] = "HILLSBURG-ROBLIN-SHELL RIVER"

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])

gdf

In [ ]:
gdf.explore()

In [ ]:
gdf['RM']=gdf['RM'].astype('string')
df['RM']=df['RM'].astype('string')

In [ ]:
gdf = gdf.reset_index()

In [ ]:
df

In [ ]:
# SQL inner join
df_gdf=pd.merge(gdf, df, on='RM', how='inner')  ##### tem algo errado nesse join porque o resultado tem so province SK

In [ ]:
df_gdf

In [ ]:
print('RM Before merging', gdf['RM'].nunique())
print('RM After merging',df['RM'].nunique())
print('RM After merging',df_gdf['RM'].nunique())

In [ ]:
ds=df_gdf.copy()

In [ ]:
stats = ds.groupby (["RM"]).agg({"Oats":["mean","std","min","max"]})
stats.to_csv ('/Users/robertotabosa/Desktop/Stream3/stats.csv')
stats = stats.rename(columns={"O___a___t___s_______m___a___x": "Oats_max"})
#ds.loc[ds["Province"] == "MB"]    # SK or MB
#stdSoil = ds.groupby('Province')['Province'].count()
print (stats)


In [ ]:
# How replace Nan for 0

ds['Oats'] = ds['Oats'].fillna(0)   #-> nao senti mais necessidade de utilizar esse codigo


## GIS Visualization

### Explore function

In [ ]:
ds

In [ ]:
ax = ds.loc[ds['Year'] >= 2003].plot(column='Oats', legend=True, cmap='Greens', figsize=(20, 20),scheme="quantiles")

# Add web map tiles
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

plt.show()
title_html = '''
                 <h3 align="center" style="font-size:30px; color:Green;"><b> Oats Yield 2003 - 2022 </b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.save('/Users/robertotabosa/Desktop/Stream3/Oats.html')

In [ ]:
ds.plot(column='Oats', 
                                 legend=True,  
                                 cmap='Greens')

## Tableau

## Aggregations

In [ ]:
# entendendo o resultado dos graficos abaixo
t = df[df["Year"]>=2003]

df.to_csv("/Users/robertotabosa/Desktop/Stream3/df.csv")

In [ ]:
df_03_22=df.loc[df['Year']>=2003].sort_values(['RM', 'Year'])

In [ ]:
df_03_22.to_csv("/Users/robertotabosa/Desktop/Stream3/df_03_22.csv")

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(5, 4, figsize=(20, 25), sharey=True)
years_to_plot = df_03_22['Year'].unique()
plt.rc('xtick', labelsize=8)    # fontsize of the tick labels
plt.rc('ytick', labelsize=8)
plt.rc('legend', fontsize=7, frameon=False)

for i, year in enumerate(years_to_plot):
    ax = axes[i//4, i%4]
    merged_df = pd.merge(gdf, df_03_22.loc[df_03_22['Year'] == year], on='RM')
    merged_df.plot(column='Oats', cmap='RdYlGn', legend=True, ax=ax, scheme="quantiles")
    ax.set_title(f'Oats Yield in {year}', color='Blue', size=12)


plt.tight_layout()
plt.show()

In [ ]:
df_agg = stats
df_agg= df_agg.dropna() # Dropped all whith null

In [ ]:
df_agg.info()

## K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans 

# Rename columns with a single level index
df_agg.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in df_agg.columns]

# Select the columns with the correct MultiIndex format
df_agg_can = df_agg[['Oats_mean', 'Oats_std']]

# Let's define our features
X = df_agg_can.copy()

from sklearn.metrics import silhouette_score
n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  # number of clusters
clusters_inertia = []  # inertia of clusters
s_scores = []  # silhouette scores

for n in n_clusters:
    KM = KMeans(n_clusters=n, init='k-means++', n_init=10).fit(X)
    clusters_inertia.append(KM.inertia_)  # data for the elbow method
    silhouette_avg = silhouette_score(X, KM.labels_)
    s_scores.append(silhouette_avg)  # data for the silhouette score method



## Elbow Metod

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, clusters_inertia, 'o-', color='blue', label='Elbow Method')
ax.set_title("Elbow Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Clusters Inertia")
ax.axvline(4, ls="--", c="red")
ax.axvline(5, ls="--", c="red")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()

## Silhouette Score

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, s_scores, 's-', color='green', label='Silhouette Score Method')
ax.set_title("Silhouette Score Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Silhouette Score")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# ---- New code:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42, n_init=10)

# Fit and predict the clusters
df_agg_can['Clusters_4'] = kmeans.fit_predict(df_agg_can[['Oats_mean', 'Oats_std']])

# Use .loc to avoid the warning
df_agg_can.loc[:, 'Clusters_4'] = df_agg_can['Clusters_4']


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42, n_init=10)

# Fit and predict the clusters
df_agg_can['Clusters_6'] = kmeans.fit_predict(df_agg_can[['Oats_mean', 'Oats_std']])

# Use .loc to avoid the warning
df_agg_can.loc[:, 'Clusters_6'] = df_agg_can['Clusters_6']

In [ ]:
sb.scatterplot(data=df_agg_can, x='Oats_mean', y='Oats_std', hue='Clusters_4')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'blue', label = 'Centroids')
plt.title('Oats Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
df_agg_can.to_csv('/Users/robertotabosa/Desktop/Stream3/sg_agg_can.csv')

In [ ]:
sb.scatterplot(data=df_agg_can, x='Oats_mean', y='Oats_std', hue='Clusters_6')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'blue', label = 'Centroids')
plt.title('Oats Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_6', legend='True', k=6, scheme='naturalbreaks', cmap='Oranges')

In [ ]:
df_agg_can

## Ranking clusters based on Mean

In [ ]:
df_agg_can.groupby('Clusters_6').mean()\
    .sort_values('Oats_mean')[['Oats_mean', 'Oats_std']]

In [ ]:
# Ranking based on the mean
df_agg_can['Clusters_6_ranked']=df_agg_can['Clusters_6'].replace(to_replace={
    0:1,
    5:2,
    2:3,
    4:4,
    1:5,
    3:6
})

In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_6_ranked', legend='True', k=6, scheme='naturalbreaks', cmap='Oranges')